## Canvas Map

Creates an hexagon map as a base for all research and development

Uber hexagon project:
https://nbviewer.jupyter.org/github/uber/h3-py-notebooks/blob/master/H3%20API%20examples%20on%20Urban%20Analytics.ipynb

In [4]:
%%sh
cat <<EOF > requirements.txt
h3==3.1.0
pandas>=0.23
geojson==2.4.1
folium==0.7.0
seaborn==0.9.0
area==1.1.1
statsmodels==0.9.0
shapely
descartes
geopandas
EOF

In [7]:
%%sh
pip3 install -r requirements.txt

Process is interrupted.


In [63]:
%%sh
pip3 install descartes

In [9]:
import json
import pandas as pd
from shapely.ops import cascaded_union
from pandas.io.json import json_normalize
from shapely.geometry import Point, Polygon
import geopandas as gpd
from folium import Map, Marker, GeoJson

In [2]:
with open("convertcsv.geojson") as f:
    loc_geodata = json.load(f)
    
df_subzones = pd.DataFrame(pd.json_normalize(loc_geodata['features']))
df_subzones.head(5)

,type,geometry.type,geometry.coordinates,properties.localidad
0,Feature,Point,"[-74.0571685, 4.6869486]",usaquen
1,Feature,Point,"[-74.0563848, 4.6867864]",usaquen
2,Feature,Point,"[-74.0549382, 4.6864427]",usaquen
3,Feature,Point,"[-74.0546701, 4.6863871]",usaquen
4,Feature,Point,"[-74.053883, 4.6862325]",usaquen


In [3]:
df = df_subzones.groupby('properties.localidad')['geometry.coordinates'].apply(lambda x: Polygon(x.tolist())).reset_index()

In [4]:
df

,properties.localidad,geometry.coordinates
0,antonio narino,"POLYGON ((-74.1308846 4.5936364, -74.130426099..."
1,barrios unidos,"POLYGON ((-74.0933105 4.6662185, -74.0929208 4..."
2,bosa,"POLYGON ((-74.22356689999999 4.6255475, -74.22..."
3,chapinero,"POLYGON ((-74.0683599 4.6288107, -74.067597500..."
4,ciudad bolivar,"POLYGON ((-74.2122251 4.3916744, -74.2103292 4..."
5,engativa,"POLYGON ((-74.16044890000001 4.7201089, -74.15..."
6,fontibon,"POLYGON ((-74.1769551 4.7023652, -74.1763543 4..."
7,kennedy,"POLYGON ((-74.1855841 4.6466405, -74.185573700..."
8,la candelaria,"POLYGON ((-74.0821845 4.5932599, -74.0814962 4..."
9,los martires,"POLYGON ((-74.10704699999999 4.5984312, -74.10..."


In [20]:
#Delete Sumapaz, usme y san cristobal because are big locations without much interest population
df = df.drop([4, 12, 15, 19])
df

KeyError: '[12 15 19] not found in axis'

In [16]:
from shapely.ops import cascaded_union
polygons = [df['geometry.coordinates'][0], df['geometry.coordinates'][1],
           df['geometry.coordinates'][2], df['geometry.coordinates'][3],
           df['geometry.coordinates'][5], df['geometry.coordinates'][6], 
           df['geometry.coordinates'][7], df['geometry.coordinates'][8], 
           df['geometry.coordinates'][9], df['geometry.coordinates'][10], 
           df['geometry.coordinates'][11], df['geometry.coordinates'][13], 
           df['geometry.coordinates'][14], df['geometry.coordinates'][16], 
           df['geometry.coordinates'][17],
           df['geometry.coordinates'][18]]
boundary = gpd.GeoSeries(cascaded_union(polygons))
bog_bound = boundary.to_json()
bog_bound

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[-74.2122251, 4.3916744], [-74.2120858, 4.3927944], [-74.2119291, 4.3948428], [-74.2116155, 4.3968912], [-74.210517, 4.3986246], [-74.2103604, 4.4009881], [-74.2097328, 4.4027214], [-74.2097332, 4.4046122], [-74.2086348, 4.4076061], [-74.2078502, 4.4093395], [-74.2069086, 4.4109153], [-74.2062809, 4.4121759], [-74.2054963, 4.4139093], [-74.2043176, 4.4150592], [-74.2040841, 4.4178487], [-74.2031426, 4.4198351], [-74.2017933, 4.4205123], [-74.2007579, 4.4224868], [-74.2001608, 4.42557], [-74.1993761, 4.4266731], [-74.1982773, 4.4277763], [-74.1968646, 4.4287219], [-74.1951379, 4.42951], [-74.1949812, 4.4309281], [-74.1948245, 4.4329765], [-74.1949819, 4.4354975], [-74.1938832, 4.4367582], [-74.1932554, 4.4378612], [-74.1927847, 4.4397521], [-74.192157, 4.4411703], [-74.1915294, 4.4436914], [-74.1905878, 4.4455824], [-74.1899601, 4.4476308], 

In [19]:
map_bogota = Map(location= [4.6736789,-74.0400262], zoom_start=10, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
GeoJson(
        bog_bound,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'green',
            'weight': 3,
            'fillOpacity': 0
        }, 
        name = "Subzone" 
    ).add_to(map_bogota)
map_bogota.save('map_bogota.html')
map_bogota